# 3_1 Compare performance of group model and individual models
1. Load and split training set, validation set and testing set
2. write Cross validation function
3. train and save individual models with cross-validation
4. train and save group model without cross-validation
5. print performance

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Load my models and functions
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *



In [2]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0
tf.disable_eager_execution()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


In [3]:

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)

random_seed  = 1000
batch_size = 64
device


Device:  cuda Device Count:  2 Device Name:  Tesla V100-PCIE-16GB
Torch version: 1.7.1


device(type='cuda')

## Load dataset

In [4]:
EPOCHS = 30
winmin = 3 
stridesec = 15

print_settings(winmin,stridesec, EPOCHS)

*****************************************************************

Execution Started at 12/20/2020, 19:54:18
WindowLength: 3.00 min (2700 datum)	Slide: 15 (225 datum)	Epochs:30



In [5]:
person = "lawler"
meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)



Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.10/10.10.shm
Loading File:  ../data/IndividualData/lawler-data/10.12/10.12.shm
Loading File:  ../data/IndividualData/lawler-data/10.14/10.14.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/afternoon_2hr33min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.21/10.21.shm
Loading File:  ../data/IndividualData/lawler-data/10.26/10.26.shm
Loading File:  ../data/IndividualData/lawler-data/10.28/10.28.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/evening-2hr_goodDinnerTemplate_CFAmeal/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.4/10.4.shm
Loading File:  ../data/IndividualData/lawler-dat

In [6]:

# method 1 to split train, test dataset
samples,labels =  meal_data.data_indices, meal_data.labels

# split train set and test set
train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                        y = labels, test_size = 0.2,
                                                       random_seed = random_seed)
# balance train set
trainset_labels = labels[train_indices]
train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)

# split validation set
# balanced_trainset_labels = labels[train_indices_balanced]
# train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
#                                                         y = balanced_trainset_labels, test_size = 0.2,
#                                                        random_seed = random_seed)

Train set size: 31896, with 1438 positive samples and 30458 negative samples
Test set size: 7974, with 359 positive samples and 7615 negative samples


In [7]:
a = set(train_indices)
b = set(test_indices)
a.intersection(b)

set()

In [8]:
# method 2 to split train, test dataset
# # load
# trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
# testsamples,testlabels =  meal_data_test.data_indices, meal_data_test.labels

# train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
# test_shuffledUnderSampledBalancedIndices = balance_data_indices(testlabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
# print("Train set size: %d, Test set size: %d"%(len(train_shuffledUnderSampledBalancedIndices),len(test_shuffledUnderSampledBalancedIndices)))
# train_set_balanced = torch.utils.data.Subset(meal_data_train, train_indices)
# valid_set_balanced = torch.utils.data.Subset(meal_data_train, valid_indices)
# # test_set_balanced = torch.utils.data.Subset(meal_data_test, test_shuffledUnderSampledBalancedIndices)

# train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
# valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
# # test_loader = torch.utils.data.DataLoader(test_set_balanced ,batch_size=batch_size, shuffle=True,num_workers=2)
# print("Data Loader Created")

In [9]:
train_set_balanced = torch.utils.data.Subset(meal_data, train_indices)
# valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)

test_set = torch.utils.data.Subset(meal_data, test_indices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
# valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [10]:
def cross_validation(dataset, data_indices, model,n_epochs=30,k=5, device="cpu", random_state = 1000, checkpoint_path = "./"  ):
    from sklearn.model_selection import StratifiedKFold
    
    best_val_score = 0
    overall_best_model = None
    best_fold = None
    all_loss_ls = []
    all_train_acc_ls = []
    all_valid_acc_ls = []
    data_indices = np.array(data_indices)
    
    skf = StratifiedKFold(n_splits=k)
    
    labels = dataset.labels[data_indices]
    np.random.seed(random_state)
    seeds = np.random.randint(low=0, high=1000,size=k)
    
    
    for fold_ind, (train_fold, valid_fold) in enumerate(skf.split(data_indices, labels)):
        torch.manual_seed(seeds[fold_ind])
        
        print("===========================> Running Fold: %d"%(fold_ind))
        print()
        train_indices = data_indices[train_fold]
        valid_indices = data_indices[valid_fold]
        # Train set    
        train_set_fold = torch.utils.data.Subset(dataset, train_indices)
        train_loader_fold = torch.utils.data.DataLoader(train_set_fold,batch_size=32, shuffle=True)

        # validation set
        valid_set_fold = torch.utils.data.Subset(dataset, valid_indices)
        valid_loader_fold = torch.utils.data.DataLoader(valid_set_fold,batch_size=32, shuffle=True)
          
        # Re-initialize models
        cv_model = model
        # Since I use a dynamic created layer in network, need to input a sample to initialize the model first
        cv_model.apply(weights_init)
        cv_model.to(device)
        criterion = nn.BCEWithLogitsLoss()

        optimizer = optim.Adam(cv_model.parameters(),lr=0.01,  weight_decay=0.1)
        lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',patience= 2, factor = 0.1,threshold=0.01)


        dataloader = (train_loader_fold,valid_loader_fold )
        cv_model, best_model,val_score,loss_ls, train_acc_ls, valid_acc_ls = train_model(cv_model,dataloader, optimizer, criterion, 
                                                                                      lrscheduler, device= device,
                                                                            n_epochs=n_epochs, patience = 5, l1_enabled=False,
                                                                            checkpoint_name =checkpoint_path+"cross_valid_checkpoint_"+str(fold_ind)+".pt")
        best_model.eval()
        valid_acc, recall = eval_model(best_model, valid_loader_fold,device)
        
        all_valid_acc_ls.append(valid_acc)
        
        print("Fold %d Completed"%(fold_ind))
    print("Cross Validation Completed，score is %.4f %%"%( np.mean(all_valid_acc_ls)))
    
    return all_valid_acc_ls
        

    
    

In [23]:
torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
_ = model(torch.rand((1, input_shape[0],input_shape[1])))
model.to(device)
score = cross_validation(meal_data, train_indices_balanced,model,
                n_epochs=20,k=5, device=device, random_state = 1000, checkpoint_path = "../models/torch_models/"  )

===========================> Running Fold: 0

Training set batch amounts: 72
Test set : 18
Start Training..
Epoch: 0,  Epoch_Loss: 38.4488, Train Acc: 55.3043 %, Train Recall: 0.5591, Validation Acc:  61.1111 %,  Validation Recall: 0.5000  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 6.0468, Train Acc: 62.0870 %, Train Recall: 0.6191, Validation Acc:  65.4514 %,  Validation Recall: 0.7812  
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 2.0216, Train Acc: 66.2609 %, Train Recall: 0.6583, Validation Acc:  55.3819 %,  Validation Recall: 0.8576  


Epoch: 3,  Epoch_Loss: 1.0348, Train Acc: 68.4783 %, Train Recall: 0.6922, Validation Acc:  54.8611 %,  Validation Recall: 0.1354  


Epoch: 4,  Epoch_Loss: 0.6697, Train Acc: 72.2609 %, Train Recall: 0.7183, Validation Acc:  67.5347 %,  Validation Recall: 0.6806  
Checkpoint Saved


Epoch: 5,  Epoch_Loss: 0.5622, Train Acc: 73.5217 %, Train Recall: 0.7374, Validation Acc:  70.3125 %,  Validation Recall: 0.8646  
Checkpoint Saved


Epoch: 6,  Epoch_Lo

Epoch: 16,  Epoch_Loss: 0.3548, Train Acc: 85.8322 %, Train Recall: 0.8749, Validation Acc:  78.2609 %,  Validation Recall: 0.8258  


Epoch: 17,  Epoch_Loss: 0.3596, Train Acc: 86.1365 %, Train Recall: 0.8801, Validation Acc:  78.4348 %,  Validation Recall: 0.8328  


Epoch: 18,  Epoch_Loss: 0.3574, Train Acc: 85.3977 %, Train Recall: 0.8671, Validation Acc:  78.7826 %,  Validation Recall: 0.8049  


Epoch: 19,  Epoch_Loss: 0.3534, Train Acc: 86.0061 %, Train Recall: 0.8775, Validation Acc:  80.0000 %,  Validation Recall: 0.8293  
Checkpoint Saved


Load Best Model.
Training completed
Fold 2 Completed
===========================> Running Fold: 3

Training set batch amounts: 72
Test set : 18
Start Training..
Epoch: 0,  Epoch_Loss: 25.5756, Train Acc: 56.1930 %, Train Recall: 0.5565, Validation Acc:  57.5652 %,  Validation Recall: 0.8194  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 5.8759, Train Acc: 61.3212 %, Train Recall: 0.6165, Validation Acc:  57.3913 %,  Validation Recall: 0.4722 

In [24]:
score

[76.73611111111111,
 72.17391304347827,
 80.0,
 62.08695652173913,
 67.82608695652173]

In [11]:
torch.manual_seed(random_seed)

# split validation set
balanced_trainset_labels = labels[train_indices_balanced]
train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                        y = balanced_trainset_labels, test_size = 0.2,
                                                       random_seed = random_seed)

valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# train model
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_1 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
_ = model_1(torch.rand((1, input_shape[0],input_shape[1])))
model_1.to(device)
criterion_1 = nn.BCEWithLogitsLoss()
optimizer_1 = optim.Adam(model_1.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_1, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_1, best_model_1,val_score_1,loss_ls_1, train_acc_ls_1, valid_acc_ls_1 = train_model(model_1,dataloader, optimizer_1, 
                                                                    criterion_1, lrscheduler_1, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/lawler_models/checkpoint_model_resnet.pt" )

Train set size: 2300, with 1150 positive samples and 1150 negative samples
Test set size: 576, with 288 positive samples and 288 negative samples
Training set batch amounts: 499
Test set : 9
Start Training..
Epoch: 0,  Epoch_Loss: 0.2265, Train Acc: 95.3411 %, Train Recall: 0.0035, Validation Acc:  50.0000 %,  Validation Recall: 0.0000  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.1870, Train Acc: 95.4916 %, Train Recall: 0.0000, Validation Acc:  50.0000 %,  Validation Recall: 0.0000  


Epoch: 2,  Epoch_Loss: 0.1870, Train Acc: 95.4916 %, Train Recall: 0.0000, Validation Acc:  50.0000 %,  Validation Recall: 0.0000  




KeyboardInterrupt: 

In [ ]:
best_model_1.eval()
acc, recall = eval_model(best_model_1, test_loader,device)
print("Test Accuracy:", acc)

In [33]:
person = "adam"
meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
samples,labels =  meal_data.data_indices, meal_data.labels
# split train set and test set
train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                        y = labels, test_size = 0.2,
                                                       random_seed = random_seed)
# balance train set
trainset_labels = labels[train_indices]
train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)


Loading Dataset ...
Loading File:  ../data/IndividualData/adam-data/09-22-2020/09-22-2020.shm
Loading File:  ../data/IndividualData/adam-data/10-28-2020/10-28-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-09-2020/11-09-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-11-2020/11-11-2020-1.shm
Loading File:  ../data/IndividualData/adam-data/11-12-2020/11-12-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-16-2020/11-16-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-17-2020/11-17-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-24-2020/11-24-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-25-2020/11-25-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-26-2020/11-26-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-30-2020/11-30-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-01-2020/12-01-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-02-2020/12-02-2020.shm
Loading File:  .

In [34]:
train_set_balanced = torch.utils.data.Subset(meal_data, train_indices)


test_set = torch.utils.data.Subset(meal_data, test_indices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [ ]:
torch.manual_seed(random_seed)
# split validation set
balanced_trainset_labels = labels[train_indices_balanced]
train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                        y = balanced_trainset_labels, test_size = 0.2,
                                                       random_seed = random_seed)

valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# train model
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_2 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
_ = model_2(torch.rand((1, input_shape[0],input_shape[1])))
model_2.to(device)
criterion_2 = nn.BCEWithLogitsLoss()
optimizer_2 = optim.Adam(model_2.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_2 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_2, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_2, best_model_2,val_score_2,loss_ls_2, train_acc_ls_2, valid_acc_ls_2 = train_model(model_2,dataloader, optimizer_2, 
                                                                    criterion_2, lrscheduler_2, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/adam_models/checkpoint_model_resnet.pt" )

Train set size: 1419, with 709 positive samples and 710 negative samples
Test set size: 355, with 178 positive samples and 177 negative samples
Training set batch amounts: 504
Test set : 6
Start Training..
Epoch: 0,  Epoch_Loss: 0.1595, Train Acc: 97.1786 %, Train Recall: 0.0011, Validation Acc:  49.8592 %,  Validation Recall: 0.0000  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.1258, Train Acc: 97.2499 %, Train Recall: 0.0000, Validation Acc:  49.8592 %,  Validation Recall: 0.0000  




In [32]:
from model import *
from utils import *

torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_4 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
# Since I use a dynamic created layer in network, need to input a sample to initialize the model first
_ = model_4(torch.rand((1, input_shape[0],input_shape[1])))
model_4.to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer_4 = optim.Adam(model_4.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_4 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_4, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_4, best_model_4,val_score_4,loss_ls_4, train_acc_ls_4, valid_acc_ls_4 = train_model(model_4,dataloader, optimizer_4, 
                                                                    criterion, lrscheduler_4, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/l_models/checkpoint_model_resnet.pt" )

Training set batch amounts: 499
Test set : 9
Start Training..
Epoch: 0,  Epoch_Loss: 0.2262, Train Acc: 95.3380 %, Train Recall: 0.0035 
Validation Acc:  50.0000 %,  Validation Recall: 0.0000 
Checkpoint Saved




KeyboardInterrupt: 